In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import datetime
from pyspark.sql import types
from collections import namedtuple

In [3]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/02 18:54:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext.uiWebUrl

'http://dtc-de.asia-southeast1-b.c.de-dtc-375915.internal:4040'

In [4]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_green.rdd.take(1)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 13, 38, 16), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=130, passenger_count=1, trip_distance=12.77, fare_amount=36.0, extra=0.0, mta_tax=0.5, tip_amount=2.05, tolls_amount=6.12, ehail_fee=None, improvement_surcharge=0.3, total_amount=44.97, payment_type=1, trip_type=1, congestion_surcharge=0.0)]

In [6]:
rdd = (df_green.select("lpep_pickup_datetime", "PULocationID", "total_amount").rdd)


In [7]:
start = datetime(year=2020, month=1, day=1)

In [8]:
def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [10]:
def prepare_group(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID

    amount = row.total_amount
    count = 1

    return ((hour, zone), (amount, count))


In [18]:
def calc_revenue(acc, new):
    acc_amount, acc_count = acc
    new_amount, new_count = new

    acc_amount = acc_amount + new_amount
    acc_count = acc_count + new_count

    return (acc_amount, acc_count)

In [26]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

def unwrap_reduced(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )



In [27]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [29]:
df_result = (
    rdd
    .filter(filter_outliers)
    .map(prepare_group)
    .reduceByKey(calc_revenue)
    .map(unwrap_reduced)
    .toDF(result_schema)
)


In [1]:
df_green = spark.read.parquet('data/pq/green/*/*')

NameError: name 'spark' is not defined